In [1]:
!nvidia-smi

Thu Jun 15 15:19:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/src/Notebook/
%ls

/content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/src/Notebook
'attention map.ipynb'   golfdb_val_test_xgg_inference_tcn.ipynb
 golfdb/                golfdb_xgg_CodaLab.ipynb
 golfdb_demo.ipynb      golfdb_xgg_inference_best.ipynb
 golfdb_f.ipynb         golfdb_xgg_inference_tcn.ipynb


In [4]:
!git clone https://github.com/wmcnally/golfdb.git
%cd golfdb/
%ls

fatal: destination path 'golfdb' already exists and is not an empty directory.
/content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/src/Notebook/golfdb
CodaLab_val.csv
data/
dataloader_custom.py
dataloader.py
eval.py
golfdb/
golfdb_G3_fold1_iter10000_val_test_X.csv
golfdb_G3_fold2_iter3000_val_test_X.csv
golfdb_G3_fold5_iter3000_val_test_X.csv
golfdb.png
golfdb_tcn_fold4_iter10000_X.csv
golfdb_tcn_fold4_iter1000_X.csv
golfdb_tcn_fold4_iter2000_X.csv
golfdb_tcn_fold4_iter3000_X.csv
golfdb_tcn_fold4_iter4000_X.csv
golfdb_tcn_fold4_iter5000_X.csv
golfdb_tcn_fold4_iter6000_X.csv
golfdb_tcn_fold4_iter7000_X.csv
golfdb_tcn_fold4_iter8000_X.csv
golfdb_tcn_fold4_iter9000_X.csv
golfdb_tcn_fold5_iter10000_X.csv
golfdb_tcn_fold5_iter2000_X.csv
golfdb_tcn_fold5_iter3000_X.csv
golfdb_tcn_fold5_iter4000_X.csv
golfdb_tcn_fold5_iter5000_X.csv
golfdb_tcn_fold5_iter6000_X.csv
golfdb_tcn_fold5_iter7000_X.csv
golfdb_tcn_fold5_iter8000_X.csv


## Import packages

In [5]:
import argparse
import os
import cv2
import csv
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from eval import ToTensor, Normalize
from model_custom import EventDetector
import torch.nn.functional as F

## functions

In [6]:
event_names = {
    0: 'HitFrame'
}


class SampleVideo(Dataset):
    def __init__(self, path, input_size=160, transform=None):
        self.path = path
        self.input_size = input_size
        self.transform = transform

    def __len__(self):
        return 1

    def __getitem__(self, idx):
        cap = cv2.VideoCapture(self.path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print('frame_count =', frame_count)
        frame_size = [cap.get(cv2.CAP_PROP_FRAME_HEIGHT), cap.get(cv2.CAP_PROP_FRAME_WIDTH)]
        ratio = self.input_size / max(frame_size)
        new_size = tuple([int(x * ratio) for x in frame_size])
        delta_w = self.input_size - new_size[1]
        delta_h = self.input_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)

        #print(new_size)
        # preprocess and return frames
        images = []
        for pos in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
            _, img = cap.read()

            img = img[:, 280:1280-280]
            resized = cv2.resize(img, (180, 180), interpolation=cv2.INTER_AREA)    ################################ 1. test-size ################################

            #resized = cv2.resize(img, (new_size[1], new_size[0]))
            b_img = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT,
                                       value=[0.406 * 255, 0.456 * 255, 0.485 * 255])  # ImageNet means (BGR)

            b_img_rgb = cv2.cvtColor(b_img, cv2.COLOR_BGR2RGB)
            images.append(b_img_rgb)
        cap.release()
        labels = np.zeros(len(images)) # only for compatibility with transforms
        sample = {'images': np.asarray(images), 'labels': np.asarray(labels)}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [7]:
%ls /content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/datasets/CodaLab/testdata_track1/    ################################ 2. val-folder ################################

00170_xgg.mp4  00216_xgg.mp4  00262_xgg.mp4  00308_xgg.mp4  00354_xgg.mp4
00171_xgg.mp4  00217_xgg.mp4  00263_xgg.mp4  00309_xgg.mp4  00355_xgg.mp4
00172_xgg.mp4  00218_xgg.mp4  00264_xgg.mp4  00310_xgg.mp4  00356_xgg.mp4
00173_xgg.mp4  00219_xgg.mp4  00265_xgg.mp4  00311_xgg.mp4  00357_xgg.mp4
00174_xgg.mp4  00220_xgg.mp4  00266_xgg.mp4  00312_xgg.mp4  00358_xgg.mp4
00175_xgg.mp4  00221_xgg.mp4  00267_xgg.mp4  00313_xgg.mp4  00359_xgg.mp4
00176_xgg.mp4  00222_xgg.mp4  00268_xgg.mp4  00314_xgg.mp4  00360_xgg.mp4
00177_xgg.mp4  00223_xgg.mp4  00269_xgg.mp4  00315_xgg.mp4  00361_xgg.mp4
00178_xgg.mp4  00224_xgg.mp4  00270_xgg.mp4  00316_xgg.mp4  00362_xgg.mp4
00179_xgg.mp4  00225_xgg.mp4  00271_xgg.mp4  00317_xgg.mp4  00363_xgg.mp4
00180_xgg.mp4  00226_xgg.mp4  00272_xgg.mp4  00318_xgg.mp4  00364_xgg.mp4
00181_xgg.mp4  00227_xgg.mp4  00273_xgg.mp4  00319_xgg.mp4  00365_xgg.mp4
00182_xgg.mp4  00228_xgg.mp4  00274_xgg.mp4  00320_xgg.mp4  00366_xgg.mp4
00183_xgg.mp4  00229_xgg.mp4  00275_xg

In [8]:
seq_length = 64    ################################ 3. seq-length ################################


valPath = "/content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/datasets/CodaLab/testdata_track1/"    ################################ 4. val-path ################################
valFolders = os.listdir(valPath)
valFolders = sorted(valFolders)

valVideoPath = []
valVideos = []
"""
for folder in valFolders:
    fileName = os.listdir(valPath + folder + '/')
    for fn in fileName:
        if fn[-1] == '4':
            videoPath = valPath + folder + '/' + fn
            valVideoPath.append(videoPath)
            valVideos.append(fn)
"""
for fn in valFolders:
    videoPath = valPath + fn
    valVideoPath.append(videoPath)
    valVideos.append(fn)

#print(valVideos)


predictList = []

data = []

for vp in valVideoPath:

    #vp = '/home/yuhsi/Badminton/data/part1/val/00004/00004.mp4'    # debug

    print('Preparing video: {}'.format(vp))

    ds = SampleVideo(vp, transform=transforms.Compose([ToTensor(),
                                Normalize([0.485, 0.456, 0.406],
                                        [0.229, 0.224, 0.225])]))

    dl = DataLoader(ds, batch_size=1, shuffle=False, drop_last=False)

    model = EventDetector(pretrain=True,
                        width_mult=1.,
                        lstm_layers=1,
                        lstm_hidden=256,
                        bidirectional=True,
                        dropout=False)

    try:
        save_dict = torch.load('/content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/weights/golfdb_G3/models/fold5_swingnet_3000.pth.tar')    ################################ 5. trained-weight ################################
    except:
        print("Model weights not found. Download model weights and place in 'models' folder. See README for instructions")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using device:', device)
    model.load_state_dict(save_dict['model_state_dict'])
    model.to(device)
    model.eval()
    print("Loaded model weights")

    print('Testing...')
    for sample in dl:
        images = sample['images']
        # full samples do not fit into GPU memory so evaluate sample in 'seq_length' batches
        batch = 0
        while batch * seq_length < images.shape[1]:
            if (batch + 1) * seq_length > images.shape[1]:
                image_batch = images[:, batch * seq_length:, :, :, :]
            else:
                image_batch = images[:, batch * seq_length:(batch + 1) * seq_length, :, :, :]
            logits = model(image_batch.cuda())
            if batch == 0:
                probs = F.softmax(logits.data, dim=1).cpu().numpy()
            else:
                probs = np.append(probs, F.softmax(logits.data, dim=1).cpu().numpy(), 0)
            batch += 1


    probList = probs[:, 0]
    #print(probList)

    #events = np.argmax(probs, axis=0)[:-1]
    #print('Predicted event frames: {}'.format(events))

    eventList = []
    #thres = np.average(probList)
    higher_q = np.quantile(probList, 0.8, interpolation='higher')    ################################ 6. filter-thres ################################
    thres = higher_q
    #print(thres, higher_q)
    for pi in range(len(list(probList))):
        if list(probList)[pi] > thres:
            eventList.append([pi])
    print('Predicted event frames: {}'.format(eventList))

    # filter the events
    filteredList = [[]]
    for i in range(len(eventList)-1):
        if np.abs(eventList[i+1][0] - eventList[i][0]) == 1:
            filteredList[-1].append(eventList[i][0])
        else:
            filteredList[-1].append(eventList[i][0])
            filteredList.append([])
    print('filteredList =', filteredList)

    new_eventList = []
    for l in filteredList:
        if len(l) >= 3:    ################################ 7. filter-len ################################
            ll = []
            for i in range(len(l)):
                ll.append(probs[l[i]][0])
            e = l[np.argmax(ll)]
            #print(e)
            new_eventList.append([e])
    print('new_eventList =', new_eventList)


    eventList = new_eventList


    ss = 1
    vn_ = vp.split('/')[-1]
    vn = vn_.split('_')[0] + '.mp4'
    for ie in range(len(eventList)):
        #print(vn, eventList[ie][0])
        if ie == len(eventList) - 1:
            data.append([vn, ss, eventList[ie][0], 'X',
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'X'])
        else:
            data.append([vn, ss, eventList[ie][0], 'X',
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'X'])


        # visualize
        """
        cap = cv2.VideoCapture(vp)
        confidence = []
        events = eventList[ie]
        for i, e in enumerate(events):
            confidence.append(probs[e, i])
        print('Condifence: {}'.format([np.round(c, 3) for c in confidence]))

        for i, e in enumerate(events):
            cap.set(cv2.CAP_PROP_POS_FRAMES, e)
            _, img = cap.read()
            cv2.putText(img, '{:.3f}'.format(confidence[i]), (20, 20), cv2.FONT_HERSHEY_DUPLEX, 0.75, (0, 0, 255))
            cv2.imshow(event_names[i], img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        """

        ss += 1
    #break


# write csv file
header = ['VideoName', 'ShotSeq', 'HitFrame', 'Hitter', 'RoundHead', 'Backhand', 'BallHeight',
          'LandingX', 'LandingY', 'HitterLocationX', 'HitterLocationY',
          'DefenderLocationX', 'DefenderLocationY', 'BallType', 'Winner']

with open('CodaLab_testdata_track1.csv', 'w', encoding='UTF8', newline='') as f:    ################################ 8. output-name ################################
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

Preparing video: /content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/datasets/CodaLab/testdata_track1/00170_xgg.mp4
Using device: cuda
Loaded model weights
Testing...
frame_count = 393


<ipython-input-8-a768d76e3a3b>:88: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they. (Deprecated NumPy 1.22)
  higher_q = np.quantile(probList, 0.8, interpolation='higher')    ################################ 6. filter-thres ################################


Predicted event frames: [[48], [49], [50], [51], [52], [53], [54], [55], [56], [85], [86], [87], [88], [111], [112], [113], [114], [115], [116], [117], [135], [136], [137], [138], [139], [140], [141], [142], [181], [182], [183], [184], [198], [199], [200], [201], [202], [203], [206], [207], [218], [219], [220], [221], [222], [223], [224], [245], [259], [260], [261], [262], [263], [264], [292], [293], [308], [309], [310], [311], [312], [313], [314], [315], [322], [323], [324], [325], [326], [327], [336], [337], [338], [339], [340], [341], [383], [392]]
filteredList = [[48, 49, 50, 51, 52, 53, 54, 55, 56], [85, 86, 87, 88], [111, 112, 113, 114, 115, 116, 117], [135, 136, 137, 138, 139, 140, 141, 142], [181, 182, 183, 184], [198, 199, 200, 201, 202, 203], [206, 207], [218, 219, 220, 221, 222, 223, 224], [245], [259, 260, 261, 262, 263, 264], [292, 293], [308, 309, 310, 311, 312, 313, 314, 315], [322, 323, 324, 325, 326, 327], [336, 337, 338, 339, 340, 341], [383], []]
new_eventList = [[53